In [1]:
import os, shutil
import tensorflow as tf
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import load_model
import keras
from keras.utils.vis_utils import plot_model
import numpy as np
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import time

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Create directories

In [2]:
original_dataset_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/train_pics'

In [3]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_small/'
#os.mkdir(base_dir)

In [4]:
train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)

In [5]:
train_jump_dir = os.path.join(train_dir, 'jump')
#os.mkdir(train_jump_dir)
train_nojump_dir = os.path.join(train_dir, 'nojump')
#os.mkdir(train_nojump_dir)

In [6]:
validation_jump_dir = os.path.join(validation_dir, 'jump')
#os.mkdir(validation_jump_dir)
validation_nojump_dir = os.path.join(validation_dir, 'nojump')
#os.mkdir(validation_nojump_dir)

In [7]:
test_jump_dir = os.path.join(test_dir, 'jump')
#os.mkdir(test_jump_dir)
test_nojump_dir = os.path.join(test_dir, 'nojump')
#os.mkdir(test_nojump_dir)

In [8]:
print("total training jump images:", len(os.listdir(train_jump_dir)))
print("total training nojump images:", len(os.listdir(train_nojump_dir)))
print("total validation jump images:", len(os.listdir(validation_jump_dir)))
print("total validation nojump images:", len(os.listdir(validation_nojump_dir)))
print("total test jump images:", len(os.listdir(test_jump_dir)))
print("total test nojump images:", len(os.listdir(test_nojump_dir)))

total training jump images: 434
total training nojump images: 395
total validation jump images: 65
total validation nojump images: 81
total test jump images: 24
total test nojump images: 197


In [2]:
model = models.Sequential()
model.add(layers.Conv2D(32, (10,10), activation = "relu", input_shape=(75, 250, 3)))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Conv2D(64, (10,10), activation = "relu"))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation = "sigmoid"))

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 66, 241, 32)       9632      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 80, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 71, 64)        204864    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 23, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5888)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               3015168   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total para

In [7]:
plot_model(model, "model_1.png", show_shapes=True, show_layer_names=True)

In [23]:
model.compile(loss="binary_crossentropy",
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])

In [24]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(75, 250),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(75, 250),
        batch_size=10,
        class_mode='binary')

Found 829 images belonging to 2 classes.
Found 146 images belonging to 2 classes.


In [25]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 75, 250, 3)
labels batch shape: (20,)


In [28]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=150,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/5
150/150 [==============================] - 174s 1s/step - loss: 0.1086 - acc: 0.9579 - val_loss: 0.1190 - val_acc: 0.9793
Epoch 2/5
150/150 [==============================] - 170s 1s/step - loss: 0.0888 - acc: 0.9650 - val_loss: 0.1172 - val_acc: 0.9652
Epoch 3/5
150/150 [==============================] - 170s 1s/step - loss: 0.0742 - acc: 0.9710 - val_loss: 0.1375 - val_acc: 0.9713
Epoch 4/5
150/150 [==============================] - 175s 1s/step - loss: 0.0618 - acc: 0.9746 - val_loss: 0.1421 - val_acc: 0.9711
Epoch 5/5
150/150 [==============================] - 182s 1s/step - loss: 0.0474 - acc: 0.9813 - val_loss: 0.1375 - val_acc: 0.9754


In [29]:
history.history

{'acc': [0.9578698951265087,
  0.9644790192579545,
  0.970677446587784,
  0.9746278703656345,
  0.9811257117116817],
 'loss': [0.10875827293729401,
  0.08964133916986056,
  0.07482309369081559,
  0.06189622275477101,
  0.04790713899572078],
 'val_acc': [0.9793388397733042,
  0.9651639310062908,
  0.9713114697913654,
  0.971074374500385,
  0.9754098316685098],
 'val_loss': [0.11896656453320834,
  0.1171804571807667,
  0.13754523269732896,
  0.1420893225432104,
  0.13753560333277506]}

In [30]:
os.chdir("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/")
model.save('jump_nojump_v4.h5')

In [31]:
os.chdir("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/")
model = load_model('jump_nojump_v4.h5')

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(75, 250),
        batch_size=221,
        class_mode='binary',
        shuffle = False)


Found 221 images belonging to 2 classes.


In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

In [36]:
preds = model.predict_generator(test_generator, steps = 1)

In [37]:
preds_r = preds.round()
np.unique(preds_r, return_counts=True)

(array([0., 1.], dtype=float32), array([ 17, 204], dtype=int64))

In [38]:
preds_r[1:25]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [39]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [29]:
file = "data_small/test/jump/direction_up171.png"

In [40]:
start = time.time()
file = "data_small/test/jump/direction_up338.png"
img = image.load_img(file, target_size = (250, 75))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

model.predict(img_tensor)[0,0]
end = time.time()
print(end - start)

0.04586172103881836
